In [1]:
import fastai
from fastai.vision import *
from fastai.callbacks import *
from fastai.vision.gan import *

In [2]:
path = untar_data(URLs.PETS)
path_hr = path/'images'
path_lr = path/'crappy'

## Crappified data
Generate low resolution images

In [3]:
from PIL import Image, ImageDraw, ImageFont

In [4]:
def crappify(fn,i):
    dest = path_lr/fn.relative_to(path_hr)
    dest.parent.mkdir(parents=True, exist_ok=True)
    img = PIL.Image.open(fn)
    targ_sz = resize_to(img, 96, use_min=True)
    img = img.resize(targ_sz, resample=PIL.Image.BILINEAR).convert('RGB')
    w,h = img.size
    q = random.randint(10,70)
    ImageDraw.Draw(img).text((random.randint(0,w//2),random.randint(0,h//2)), str(q), fill=(255,255,255))
    img.save(dest, quality=q)

Uncomment the first time you run this notebook.

In [5]:
#il = ImageItemList.from_folder(path_hr)
parallel(crappify, il.items)

For gradual resizing we can change the commented line here.

In [21]:
bs,size=32, 128
# bs,size = 24,160
# bs,size = 8,256

bs,size=4, 128
arch = models.resnet34

## Pre-train generator
Now let's pretrain the generator.

In [22]:
arch = models.resnet34
src = ImageImageList.from_folder(path_lr).random_split_by_pct(0.1, seed=42)

In [23]:
def get_data(bs,size):
    data = (src.label_from_func(lambda x: path_hr/x.name)
           .transform(get_transforms(max_zoom=2.), size=size, tfm_y=True)
           .databunch(bs=bs).normalize(imagenet_stats, do_y=True))

    data.c = 3
    return data

In [24]:
data_gen = get_data(bs,size)

In [25]:
data_gen.show_batch(4)

StopIteration: 